# Who Will Pay Extra For Guac?
Chipotle: Mexican Grill has had a tough time recouping from the virus outbreaks in 2015 but with there stock now fully rebounded the ceo, [Brian Niccol, has said the he wants to double the amount of locations](https://www.bloomberg.com/news/articles/2019-03-14/chipotle-becomes-another-boring-chain-and-that-s-a-good-thing). The chain currently operates 2,473 loactions in 47 states. Those states being Alaska, Hawaii, and South Dakota. Using the [SafeGraph Open Census Data](https://docs.safegraph.com/docs/open-census-data) along with the [Safegraph Consumer and Visitor Insights for Neighborhoods Data](https://www.safegraph.com/neighborhood-demographics-consumer-insights) will enable my analysis of popular brands for over 220,000 neighborhoods in America along with demographic information from the US census. My goal is to identify factors that make certain Chipotle locations more succesful than others and find new locations that would capitilize on these insights.

In [1]:
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly.offline
import glob
import re
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)



## Starting with the chipotle patterns
To start off I want to look at the patterns data for anything where Chipotle is mentioned.
The patterns data columns are formatted in the follwing order

- census_block_group: The unique 12-digit FIPS code for the Census Block Group
- date_range_start: Start time for measurement period as a timestamp in UTC seconds
- date_range_end: End time for measurement period as a timestamp in UTC seconds.
- raw_visit_count: Number of visits seen by our panel to this CGB during the date range
- visitor_home_count: This column lists all the origin home CBGs for devices that visited a destination in the CBG listed in the column census_block_group (the destination CBG). The number mapped to each home CBG indicates the number of visitors observed from this home CBG that visited census_block_group during this time period. Home CBGs with less than 50 visitors to census_block_group are not included.
- visitor_work_cbgs: This column lists all the work-location CBGs for devices that visited a destination in the CBG listed in the column census_block_group (the destination CBG). The number mapped to each work CBG indicates the number of visitors observed with this work CBG that visited census_block_group during this time period. Work CBGs with less than 50 visitors to census_block_group are not include
- distance_from_home: Median distance from home traveled to CBG by visitors (of visitors whose home we have identified) in meters
- related_same_day_brand: Brands that the visitors to this CBG visited on the same day as their visit to the CBG where customer overlap differs by at least 5% from the SafeGraph national average to these brands. Order by strength of difference and limited to top ten brands.
- related_same_month_brand: Brands that the visitors to this CBG visited on the same month as their visit to the CBG where customer overlap differs by at least 5% from the SafeGraph national average. Order by strength of difference and limited to top ten brands.
- top_brands: A list of the the top brands visited in the CBG during the time period. Limited to top 10 brands.
- popularity_by_hour: A mapping of hour of the day to the number of visits in each hour over the course of the date range in local time.
- popularity_by_day: A mapping of day of week to the number of visits on each day (local time) in the course of the date range.

In [2]:
patterns_df = pd.read_csv('cbg_patterns.csv', parse_dates=True, converters={'census_block_group': lambda x: f"{x:12}"}) #used converter to maintain the zero-padded number format
pat = patterns_df

In [3]:
patterns_df.head()

,census_block_group,date_range_start,date_range_end,raw_visit_count,raw_visitor_count,visitor_home_cbgs,visitor_work_cbgs,distance_from_home,related_same_day_brand,related_same_month_brand,top_brands,popularity_by_hour,popularity_by_day
0,010059508001,1538352000,1541030400,75122.0,18314.0,"{""010059501003"":127,""010059509001"":111,""010059...","{""010059501003"":109,""010810407002"":62,""0108104...",194724.0,"[""Chick-fil-A"",""mcdonalds"",""Marathon Petroleum...","[""walmart"",""mcdonalds"",""Dollar General"",""Chick...","[""CrossFit"",""Health Mart"",""Coldwell Banker""]","[2617,2457,2403,2519,2646,3007,3886,7566,5508,...","{""Monday"":12000,""Tuesday"":12224,""Wednesday"":10..."
1,010090506011,1538352000,1541030400,95649.0,38942.0,"{""010730113021"":210,""010090506022"":205,""010090...","{""010890111001"":271,""010730045001"":269,""010439...",120587.0,"[""Shell Oil"",""mcdonalds"",""Chick-fil-A"",""Chevron""]","[""walmart"",""mcdonalds"",""Shell Oil"",""Chick-fil-...",[],"[6556,6325,6222,6355,6586,7350,8568,8099,7378,...","{""Monday"":12125,""Tuesday"":12984,""Wednesday"":12..."
2,010479572002,1538352000,1541030400,14009.0,3039.0,"{""010479567011"":67,""010479567021"":60}","{""010479567021"":52}",67774.0,"[""Dollar General""]","[""walmart"",""Dollar General"",""mcdonalds"",""Chevr...","[""Dollar General""]","[807,790,796,786,851,951,1134,1797,1355,1241,1...","{""Monday"":2314,""Tuesday"":2340,""Wednesday"":2195..."
3,010690404001,1538352000,1541030400,128169.0,25418.0,"{""010690402013"":370,""010690402011"":322,""010690...","{""010690402024"":313,""010690415004"":203,""010450...",42684.0,"[""Chick-fil-A"",""Sam's Club"",""Dollar General"",""...","[""walmart"",""Dollar General"",""mcdonalds"",""Marat...","[""Chick-fil-A"",""Sam's Club"",""Olive Garden"",""mc...","[2121,1828,1784,1704,1861,2373,3730,7497,7093,...","{""Monday"":21141,""Tuesday"":21143,""Wednesday"":17..."
4,010730112101,1538352000,1541030400,51453.0,9499.0,"{""010090507001"":183,""010730113021"":167,""010730...","{""010730045001"":140,""010730027001"":123,""010730...",18878.0,"[""Chevron"",""Daylight Donuts"",""walmart""]","[""walmart"",""Chevron"",""Dollar General"",""Shell O...","[""Chevron"",""CrossFit""]","[3804,3716,3686,3672,3735,4115,4855,5946,4526,...","{""Monday"":8402,""Tuesday"":8414,""Wednesday"":8550..."


In [4]:
pat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220735 entries, 0 to 220734
Data columns (total 13 columns):
census_block_group          220735 non-null object
date_range_start            220735 non-null int64
date_range_end              220735 non-null int64
raw_visit_count             220629 non-null float64
raw_visitor_count           220629 non-null float64
visitor_home_cbgs           220735 non-null object
visitor_work_cbgs           220735 non-null object
distance_from_home          220518 non-null float64
related_same_day_brand      220735 non-null object
related_same_month_brand    220735 non-null object
top_brands                  220735 non-null object
popularity_by_hour          220735 non-null object
popularity_by_day           220735 non-null object
dtypes: float64(3), int64(2), object(8)
memory usage: 21.9+ MB


In [5]:
chipotle_df = pat[(pat['top_brands'].str.contains('Chipotle')) | (pat['related_same_day_brand'].str.contains('Chipotle')) | (pat['related_same_month_brand'].str.contains('Chipotle'))]
chipotle_df.head()

,census_block_group,date_range_start,date_range_end,raw_visit_count,raw_visitor_count,visitor_home_cbgs,visitor_work_cbgs,distance_from_home,related_same_day_brand,related_same_month_brand,top_brands,popularity_by_hour,popularity_by_day
54,080050056301,1538352000,1541030400,15610.0,3915.0,"{""080050056302"":82,""080050056303"":65}","{""080050056262"":134,""080050068561"":56}",6214.0,"[""King Soopers"",""Target"",""starbucks""]","[""King Soopers"",""starbucks"",""Target"",""mcdonald...",[],"[925,911,912,896,896,905,1108,1553,1695,1174,9...","{""Monday"":2585,""Tuesday"":2694,""Wednesday"":2664..."
62,110010001001,1538352000,1541030400,141721.0,22818.0,"{""110010009013"":242,""110010007011"":164,""110010...","{""110010107001"":537,""110010058001"":286,""110010...",8859.0,"[""Unleashed by Petco"",""starbucks""]","[""starbucks"",""CVS"",""SUBWAY"",""mcdonalds"",""7-Ele...","[""Unleashed by Petco"",""LEDO Pizza"",""Exxon Mobi...","[2904,2352,2111,1985,2039,2316,4025,8635,13410...","{""Monday"":21102,""Tuesday"":24103,""Wednesday"":26..."
195,360470557002,1538352000,1541030400,57948.0,20233.0,"{""360470555001"":176,""360610022011"":62}",{},9500.0,[],"[""Dunkin' Donuts"",""starbucks"",""BP"",""mcdonalds""...","[""The Bar Method""]","[4125,3653,3344,2983,2632,2541,3268,4204,4618,...","{""Monday"":7778,""Tuesday"":8069,""Wednesday"":8703..."
242,390852012003,1538352000,1541030400,20208.0,5875.0,"{""390852012002"":90,""390852013003"":69,""39085201...","{""390852013002"":80,""390852065002"":71,""39035194...",10643.0,"[""Giant Eagle"",""Heinen's Grocery Stores""]","[""Giant Eagle"",""mcdonalds"",""Speedway"",""walmart...","[""Heinen's Grocery Stores"",""SUBWAY"",""Bruegger's""]","[543,450,437,416,402,523,903,1629,1578,1242,14...","{""Monday"":3131,""Tuesday"":3233,""Wednesday"":3266..."
286,481130006031,1538352000,1541030400,20953.0,8395.0,"{""481130006015"":65,""481130004051"":56,""48113000...","{""481130100001"":137,""481130100002"":87,""4811300...",16293.0,"[""Bob's Steak & Chop"",""Chipotle Mexican Grill""...","[""starbucks"",""walmart"",""7-Eleven US"",""Exxon Mo...","[""Bob's Steak & Chop"",""Chipotle Mexican Grill""...","[1033,965,947,853,819,833,1024,1352,1370,1254,...","{""Monday"":3215,""Tuesday"":3066,""Wednesday"":3185..."


In [6]:
chipotle_df.shape

(3500, 13)

In [3]:
census_data_files = glob.glob('safegraph_open_census_data/data/*.csv')
census_data_files

['safegraph_open_census_data/data\\cbg_b00.csv',
 'safegraph_open_census_data/data\\cbg_b01.csv',
 'safegraph_open_census_data/data\\cbg_b02.csv',
 'safegraph_open_census_data/data\\cbg_b03.csv',
 'safegraph_open_census_data/data\\cbg_b07.csv',
 'safegraph_open_census_data/data\\cbg_b08.csv',
 'safegraph_open_census_data/data\\cbg_b09.csv',
 'safegraph_open_census_data/data\\cbg_b11.csv',
 'safegraph_open_census_data/data\\cbg_b12.csv',
 'safegraph_open_census_data/data\\cbg_b14.csv',
 'safegraph_open_census_data/data\\cbg_b15.csv',
 'safegraph_open_census_data/data\\cbg_b19.csv',
 'safegraph_open_census_data/data\\cbg_b20.csv',
 'safegraph_open_census_data/data\\cbg_b21.csv',
 'safegraph_open_census_data/data\\cbg_b22.csv',
 'safegraph_open_census_data/data\\cbg_b23.csv',
 'safegraph_open_census_data/data\\cbg_b25.csv',
 'safegraph_open_census_data/data\\cbg_b27.csv',
 'safegraph_open_census_data/data\\cbg_b99.csv',
 'safegraph_open_census_data/data\\cbg_c16.csv',
 'safegraph_open_cen

In [4]:
list_of_dfs = [pd.read_csv(census_data_file, converters={'census_block_group': lambda x: f"{x:12}"}) for census_data_file in census_data_files]

In [5]:
for i in range(1, len(list_of_dfs)):
    list_of_dfs[i] = list_of_dfs[i].iloc[:,1:]
        

In [5]:
list_of_dfs[0].head()

,census_block_group,B00001e1,B00001m1,B00002e1,B00002m1
0,010010201001,72.0,-1.0,32.0,-1.0
1,010010201002,173.0,-1.0,66.0,-1.0
2,010010202001,100.0,-1.0,43.0,-1.0
3,010010202002,167.0,-1.0,54.0,-1.0
4,010010203001,184.0,-1.0,80.0,-1.0


In [10]:
list_of_dfs[1].head()

,B01001e1,B01001m1,B01001e2,B01001m2,B01001e3,B01001m3,B01001e4,B01001m4,B01001e5,B01001m5,...,B01002He3,B01002Hm3,B01002Ie1,B01002Im1,B01002Ie2,B01002Im2,B01002Ie3,B01002Im3,B01003e1,B01003m1
0,745,226,356,121,33,32,46,39,4,8,...,33.6,5.9,NaN,NaN,NaN,NaN,NaN,NaN,745,226
1,1265,232,639,139,3,6,6,9,53,36,...,45.2,4.4,NaN,NaN,NaN,NaN,NaN,NaN,1265,232
2,960,261,534,176,0,11,82,71,90,62,...,37.2,9.2,NaN,NaN,NaN,NaN,NaN,NaN,960,261
3,1236,227,634,136,28,21,47,39,33,27,...,42.9,2.4,NaN,NaN,NaN,NaN,NaN,NaN,1236,227
4,2364,378,1125,229,20,24,166,92,92,72,...,37.9,9.1,27.2,20.1,27.1,25.7,28.1,15.6,2364,378


In [6]:
census_combined_df = pd.concat(list_of_dfs, ignore_index=False, axis=1)
ccd = census_combined_df
ccd.head()

,census_block_group,B00001e1,B00001m1,B00002e1,B00002m1,B01001e1,B01001m1,B01001e2,B01001m2,B01001e3,...,B24080e17,B24080m17,B24080e18,B24080m18,B24080e19,B24080m19,B24080e20,B24080m20,B24080e21,B24080m21
0,010010201001,72.0,-1.0,32.0,-1.0,745,226,356,121,33,...,6,9,0,11,6,9,16,24,0,11
1,010010201002,173.0,-1.0,66.0,-1.0,1265,232,639,139,3,...,14,23,32,17,24,16,5,7,0,11
2,010010202001,100.0,-1.0,43.0,-1.0,960,261,534,176,0,...,0,11,39,39,0,11,0,11,0,11
3,010010202002,167.0,-1.0,54.0,-1.0,1236,227,634,136,28,...,6,10,29,28,0,11,50,69,0,11
4,010010203001,184.0,-1.0,80.0,-1.0,2364,378,1125,229,20,...,28,26,44,29,30,36,73,68,0,11


In [12]:
ccd.shape

(220333, 7731)

In [7]:
field_descriptions = pd.read_csv('safegraph_open_census_data/metadata/cbg_field_descriptions.csv')
fd = field_descriptions
fd = fd.set_index('table_id')
print(fd.field_full_name.shape)
fd.field_full_name.head()

(7730,)


table_id
B00001e1    UNWEIGHTED SAMPLE COUNT OF THE POPULATION: Tot...
B00001m1    UNWEIGHTED SAMPLE COUNT OF THE POPULATION: Tot...
B00002e1    UNWEIGHTED SAMPLE HOUSING UNITS: Total: Housin...
B00002m1    UNWEIGHTED SAMPLE HOUSING UNITS: Total: Housin...
B01001e1    SEX BY AGE: Total: Total population -- (Estimate)
Name: field_full_name, dtype: object

In [8]:
field_mapping = dict(fd['field_full_name'])

field_mapping['census_block_group'] = 'census_block_group'

columns_lst = ccd.columns.values.tolist()

mapped_columns = [field_mapping[i] for i in columns_lst]

ccd.columns = mapped_columns

In [15]:
ccd.head()

,census_block_group,UNWEIGHTED SAMPLE COUNT OF THE POPULATION: Total: Total population -- (Estimate),UNWEIGHTED SAMPLE COUNT OF THE POPULATION: Total: Total population -- (Margin of Error),UNWEIGHTED SAMPLE HOUSING UNITS: Total: Housing units -- (Estimate),UNWEIGHTED SAMPLE HOUSING UNITS: Total: Housing units -- (Margin of Error),SEX BY AGE: Total: Total population -- (Estimate),SEX BY AGE: Total: Total population -- (Margin of Error),SEX BY AGE: Male: Total population -- (Estimate),SEX BY AGE: Male: Total population -- (Margin of Error),SEX BY AGE: Male: Under 5 years: Total population -- (Estimate),...,SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Local government workers: Civilian employed population 16 years and over -- (Estimate),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Local government workers: Civilian employed population 16 years and over -- (Margin of Error),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: State government workers: Civilian employed population 16 years and over -- (Estimate),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: State government workers: Civilian employed population 16 years and over -- (Margin of Error),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Federal government workers: Civilian employed population 16 years and over -- (Estimate),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Federal government workers: Civilian employed population 16 years and over -- (Margin of Error),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Self-employed in own not incorporated business workers: Civilian employed population 16 years and over -- (Estimate),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Self-employed in own not incorporated business workers: Civilian employed population 16 years and over -- (Margin of Error),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Unpaid family workers: Civilian employed population 16 years and over -- (Estimate),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Unpaid family workers: Civilian employed population 16 years and over -- (Margin of Error)
0,010010201001,72.0,-1.0,32.0,-1.0,745,226,356,121,33,...,6,9,0,11,6,9,16,24,0,11
1,010010201002,173.0,-1.0,66.0,-1.0,1265,232,639,139,3,...,14,23,32,17,24,16,5,7,0,11
2,010010202001,100.0,-1.0,43.0,-1.0,960,261,534,176,0,...,0,11,39,39,0,11,0,11,0,11
3,010010202002,167.0,-1.0,54.0,-1.0,1236,227,634,136,28,...,6,10,29,28,0,11,50,69,0,11
4,010010203001,184.0,-1.0,80.0,-1.0,2364,378,1125,229,20,...,28,26,44,29,30,36,73,68,0,11


In [9]:
county_df = pd.read_csv('safegraph_open_census_data/metadata/cbg_fips_codes.csv', dtype=str)
cdf = county_df
cdf.head()

,state,state_fips,county_fips,county,class_code
0,AL,01,001,Autauga County,H1
1,AL,01,003,Baldwin County,H1
2,AL,01,005,Barbour County,H1
3,AL,01,007,Bibb County,H1
4,AL,01,009,Blount County,H1


In [10]:
cdf['cbg'] = cdf['state_fips'] + cdf['county_fips']
cdf = cdf.set_index('cbg')
cdf.head()

,state,state_fips,county_fips,county,class_code
cbg,,,,,
01001,AL,01,001,Autauga County,H1
01003,AL,01,003,Baldwin County,H1
01005,AL,01,005,Barbour County,H1
01007,AL,01,007,Bibb County,H1
01009,AL,01,009,Blount County,H1


In [26]:
cdf['class_code'].value_counts()

H1    3126
C7      42
H4      30
H6      26
H5      11
Name: class_code, dtype: int64

In [11]:
ccd['fips'] = ccd['census_block_group'].apply(lambda x: x[0:5])

In [12]:
state_dict = dict(cdf['state'])
ccd['state'] = ccd['fips'].map(state_dict)


In [13]:
county_dict = dict(cdf['county'])
ccd['county'] = ccd['fips'].map(county_dict)

In [14]:
ccd.tail()

,census_block_group,UNWEIGHTED SAMPLE COUNT OF THE POPULATION: Total: Total population -- (Estimate),UNWEIGHTED SAMPLE COUNT OF THE POPULATION: Total: Total population -- (Margin of Error),UNWEIGHTED SAMPLE HOUSING UNITS: Total: Housing units -- (Estimate),UNWEIGHTED SAMPLE HOUSING UNITS: Total: Housing units -- (Margin of Error),SEX BY AGE: Total: Total population -- (Estimate),SEX BY AGE: Total: Total population -- (Margin of Error),SEX BY AGE: Male: Total population -- (Estimate),SEX BY AGE: Male: Total population -- (Margin of Error),SEX BY AGE: Male: Under 5 years: Total population -- (Estimate),...,SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: State government workers: Civilian employed population 16 years and over -- (Margin of Error),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Federal government workers: Civilian employed population 16 years and over -- (Estimate),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Federal government workers: Civilian employed population 16 years and over -- (Margin of Error),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Self-employed in own not incorporated business workers: Civilian employed population 16 years and over -- (Estimate),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Self-employed in own not incorporated business workers: Civilian employed population 16 years and over -- (Margin of Error),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Unpaid family workers: Civilian employed population 16 years and over -- (Estimate),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Unpaid family workers: Civilian employed population 16 years and over -- (Margin of Error),fips,state,county
220328,721537506011,42.0,-1.0,34.0,-1.0,921,332,460,171,43,...,33,0,13,0,13,0,13,72153,PR,Yauco Municipio
220329,721537506012,150.0,-1.0,84.0,-1.0,2703,531,1353,352,142,...,52,0,13,0,13,0,13,72153,PR,Yauco Municipio
220330,721537506013,64.0,-1.0,29.0,-1.0,1195,425,583,222,0,...,13,0,13,0,13,0,13,72153,PR,Yauco Municipio
220331,721537506021,118.0,-1.0,70.0,-1.0,2005,471,926,251,68,...,32,0,13,0,13,0,13,72153,PR,Yauco Municipio
220332,721537506022,41.0,-1.0,48.0,-1.0,736,371,279,123,0,...,13,9,16,0,13,0,13,72153,PR,Yauco Municipio


In [19]:
geographic_data = pd.read_csv('safegraph_open_census_data/metadata/cbg_geographic_data.csv', converters={'census_block_group': lambda x : f"{x:12}"}) #same coverter used for zero-padded format
gd = geographic_data.set_index('census_block_group')
gd.head()

C:\Users\dinel\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning:

Both a converter and dtype were specified for column census_block_group - only the converter will be used



,amount_land,amount_water,latitude,longitude
census_block_group,,,,
010010201001,4254524,23431,32.4658291,-86.4896143
010010201002,5568295,0,32.4858487,-86.4896898
010010202001,2058380,0,32.4800823,-86.4749744
010010202002,1283506,5968,32.464435,-86.4697659
010010203001,3866515,9054,32.480175,-86.4607925


In [16]:
gd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220333 entries, 0 to 220332
Data columns (total 5 columns):
census_block_group    220333 non-null object
amount_land           220333 non-null float64
amount_water          220333 non-null int64
latitude              220333 non-null float64
longitude             220333 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 8.4+ MB


In [ ]:
def column_mapper(map_from_df, map_to_df):
    cols = list(map_from_df.columns)
    for col in cols:
        col_map = dict(map_from_df[col])
        map_to_df[col] = map_to_df['census_block_group'].map(col_map)

column_mapper(gd, ccd)

In [21]:
amount_land_dict = dict(gd['amount_land'])
ccd['amount_land'] = ccd['census_block_group'].map(amount_land_dict)

In [22]:
amount_water_dict = dict(gd['amount_water'])
ccd['amount_water'] = ccd['census_block_group'].map(amount_water_dict)

In [23]:
latitude_dict = dict(gd['latitude'])
ccd['latitude'] = ccd['census_block_group'].map(latitude_dict)

In [24]:
longitude_dict = dict(gd['longitude'])
ccd['longitude'] = ccd['census_block_group'].map(longitude_dict)

In [25]:
ccd.head()

,census_block_group,UNWEIGHTED SAMPLE COUNT OF THE POPULATION: Total: Total population -- (Estimate),UNWEIGHTED SAMPLE COUNT OF THE POPULATION: Total: Total population -- (Margin of Error),UNWEIGHTED SAMPLE HOUSING UNITS: Total: Housing units -- (Estimate),UNWEIGHTED SAMPLE HOUSING UNITS: Total: Housing units -- (Margin of Error),SEX BY AGE: Total: Total population -- (Estimate),SEX BY AGE: Total: Total population -- (Margin of Error),SEX BY AGE: Male: Total population -- (Estimate),SEX BY AGE: Male: Total population -- (Margin of Error),SEX BY AGE: Male: Under 5 years: Total population -- (Estimate),...,SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Self-employed in own not incorporated business workers: Civilian employed population 16 years and over -- (Margin of Error),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Unpaid family workers: Civilian employed population 16 years and over -- (Estimate),SEX BY CLASS OF WORKER FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER: Female: Unpaid family workers: Civilian employed population 16 years and over -- (Margin of Error),fips,state,county,amount_land,amount_water,latitude,longitude
0,010010201001,72.0,-1.0,32.0,-1.0,745,226,356,121,33,...,24,0,11,01001,AL,Autauga County,4254524,23431,32.4658291,-86.4896143
1,010010201002,173.0,-1.0,66.0,-1.0,1265,232,639,139,3,...,7,0,11,01001,AL,Autauga County,5568295,0,32.4858487,-86.4896898
2,010010202001,100.0,-1.0,43.0,-1.0,960,261,534,176,0,...,11,0,11,01001,AL,Autauga County,2058380,0,32.4800823,-86.4749744
3,010010202002,167.0,-1.0,54.0,-1.0,1236,227,634,136,28,...,69,0,11,01001,AL,Autauga County,1283506,5968,32.464435,-86.4697659
4,010010203001,184.0,-1.0,80.0,-1.0,2364,378,1125,229,20,...,68,0,11,01001,AL,Autauga County,3866515,9054,32.480175,-86.4607925
